# Storey Loss Functions Applications in Vulnerability/Loss Model Calculation

## Overview

This Jupyter Notebook outlines the workflow for applying storey loss functions (SLFs) to derive a building-class vulnerability model. Using pre-existing SLFs, the workflow focuses on quantifying expected losses at each storey and combining them into a system-level vulnerability representation.

The main goals of this notebook:

1. **Storey Loss Function Application**: For each storey and seismic demand level, calculate the expected storey losses by interpolating from the provided SLFs (derived from the "StoreyLossFunctionGeneration" demo)

2. **Aggregate Total Building Loss**: Sum the interpolated storey losses to compute the total building loss at each Intensity Measure (IM) level.

3. **Fit Loss vs. IM Model**: Fit a regression model to relate the total building loss to IM level.

4. **Condition Non-Collapse Losses**: Adjust repair-related (non-collapse) losses by the system-level collapse fragility to produce the final vulnerability model for the building class.

---

## References

[1] Papadopoulos AN, Vamvatsikos D, Kazantzi AK. Development and Application of FEMA P-58 Compatible Story Loss Functions. Earthquake Spectra. 2019;35(1):95-112. doi:10.1193/102417EQS222M
  
[2] O’Reilly, G. J., & Shahnazaryan, D. (2024). On the utility of story loss functions for regional seismic vulnerability modeling and risk assessment. Earthquake Spectra, 40(3), 1933–1955. https://doi.org/10.1177/87552930241245940

## Import Libraries ## 

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt 
from scipy.optimize import curve_fit
from scipy.stats import lognorm, genpareto

# Import the classes necessary for postprocessing and visualising storey loss functions
from openquake.vmtk.plotter import plotter
from openquake.vmtk.utilities import import_from_pkl 
from openquake.vmtk.postprocessor import postprocessor

# Initialise the plotter class
pl = plotter()

# Initialise the postprocessor class
pp = postprocessor()

## Load Required Input Data ## 

In [ ]:
# The set of drift- and acceleration-sensitive inventory of nonstructural components are compatible 
# with the FEMA P-58 database and were obtained using the "Normative Quantity Estimation Tool"
# (https://femap58.atcouncil.org/supporting-materials) assuming a residential occupancy class

input_directory = './input'

# Load the drift-sensitive nonstructural components storey loss functions
slf_drift = import_from_pkl(os.path.join(input_directory, 'slf_drift.pkl'))

# Load the acceleration-sensitive nonstructural components storey loss functions
slf_accel = import_from_pkl(os.path.join(input_directory, 'slf_accel.pkl'))

# Load the analysis dictionary containing response quantities (i.e., engineering demand parameters) such as peak storey drifts and peak floor 
# acceleration necessary for the subsequent analysis using SLFs.
# This dictionary corresponds to the application of cloud analysis on a 3 storey MDOF model whose example application is available in the 
# "CloudAnalysis" demo folder.
ansys_out = import_from_pkl(os.path.join(input_directory, 'ansys_out.pkl'))

# Load the intensity measure levels dictionary file evaluated from the "IntensityMeasureProcessing" demo 
# which contains various IMs and their quantities. This input is useful for deriving the total nonstructural loss
ims = import_from_pkl(os.path.join(input_directory, 'imls_esrm20.pkl'))

## Normalise the Imported Storey Loss Functions ## 

In [ ]:
# The SLFs we imported in the previous cell contains absolute values associated with 
# the costs of repairs with increasing levels of engineering demand parameters
# For this application, we will normalise the costs by the expected total cost of repair
# per performance groups. In this way, our SLFs will represent storey loss ratios instead 
# and will additionally the reflect the contribution of each performance group on the
# overall storey losses

# Get the maximum cost associated with repairing both drift- and acceleration-
# sensitive components from SLFs
max_value = max(slf_drift['PSD, NS']['slf'])+ max(slf_accel['PFA, NS']['slf'])

# Normalise the drift-sensitive nonstructural SLF
slf_drift_norm = [x/max_value for x in slf_drift['PSD, NS']['slf']]

# Normalise the acceleration-sensitive nonstructural SLF
slf_accel_norm = [x/max_value for x in slf_accel['PFA, NS']['slf']]

# Plot the normalise SLFs vs their corresponding EDPs
psd_range = slf_drift['PSD, NS']['edp_range'] # Peak storey drifts range used to generate the SLFs in "StoreyLossFunctionGeneration" demo
pfa_range = slf_accel['PFA, NS']['edp_range'] # Peak floor accelerations range used to generate the SLFs in "StoreyLossFunctionGeneration" demo

# Plot the results 
fig, axs = plt.subplots(1, 2, figsize=(12, 5))  # 1 row, 2 columns

# --- Left plot: Drift-sensitive ---
axs[0].plot(psd_range, slf_drift_norm)
axs[0].set_xlabel('Peak Storey Drifts [rad]')
axs[0].set_ylabel('Drift-Sensitive Storey Loss Ratio')
axs[0].grid(True)
axs[0].set_xlim([0, 0.1])
axs[0].set_ylim([0, 1.0])
axes[0].set_title("Drift-Sensitive Components")

# --- Right plot: Acceleration-sensitive ---
axs[1].plot(pfa_range, slf_accel_norm)
axs[1].set_xlabel('Peak Floor Accelerations [g]')
axs[1].set_ylabel('Acceleration-Sensitive Storey Loss Ratio')
axs[1].grid(True)
axs[1].set_xlim([0, 5.0])
axs[1].set_ylim([0, 1.0])
axes[0].set_title("Acceleration-Sensitive Components")

plt.tight_layout()
plt.show()

## Interpolate Storey Loss Ratio using Storey Loss Functions and Seismic Demands ## 

In [ ]:
# First, let us visualise the seismic demands obtained from the analysis we "supposedly" carried out

pl.plot_demand_profiles(ansys_out['peak_drift_list'], 
                        ansys_out['peak_accel_list'], 
                        ansys_out['control_nodes'], 
                        output_directory = None,
                        plot_label="seismic_demand_profiles") # The y-axis values of drift and acceleration are converted to % and g automatically by the plotter


In [ ]:
# For each ground-motion record, seismic demand profiles expressing the distribution of peak storey drifts and peak floor accelerations
# along the height of the idealised MDOF model were recorded. Therefore, for each ground-motion record, we will use the demand profiles 
# and interpolate via the SLFs, the associated storey loss

# Number of ground motion records used
n_gmrs = len(ansys_out['peak_drift_list']) 

# Number of storeys
number_storeys = len(ansys_out['control_nodes'])-1

# Number of floors 
number_floors = number_storeys + 1 

# Loss weights: # These weights assume that drift- and acceleration-sensitive components are uniformly distributed
# along the height of the building. Therefore, their contribution to the total losses is the dot product of the storey loss
# and the weights
psd_weights = [1/number_storeys]*number_storeys 

pfa_weights = [1/number_floors]*number_floors

# Create dictionary to store results for each GM record
results = {"drift_total_loss": [],
           "accel_total_loss": [],
           "total_loss": [],
           "IM": []}

# Loop over ground motion records
for i in range(n_gmrs):

    ## Process losses from drift-sensitive components
    
    # Step 1: Get the drift profile at ground motion record "i"
    current_drifts = ansys_out['peak_drift_list'][i][:,0] # The indexing corresponds to fetching the peak storey drifts of ground motion "i" in the X-direction

    # Create list to append values
    drift_storey_losses = []
    
    # Step 2: Loop over the number of storeys
    for j in range(number_storeys):

        # Step 2.1: Interpolate the loss corresponding to the peak drift value at each storey
        drift_storey_loss = np.interp(current_drifts[j], psd_range, slf_drift_norm)
        drift_storey_losses.append(drift_storey_loss)
    
    # Step 3: Get the total losses along the height of the building from drift-sensitive components
    drift_storey_losses = np.array(drift_storey_losses)
    drift_total_loss    = np.dot(psd_weights, drift_storey_losses)

    ## Process losses from acceleration-sensitive components
    
    # Step 1: Get the drift profile at ground motion record "i"
    current_accels = ansys_out['peak_accel_list'][i][:,0] # The indexing corresponds to fetching the peak floor accelerations of ground motion "i" in the X-direction

    # Create list to append values
    accel_floor_losses = []
    
    # Step 2: Loop over the number of floor
    for j in range(number_floors):

        # Step 2.1: Interpolate the loss corresponding to the peak acceleration value at each floor
        accel_floor_loss = np.interp(current_accels[j]/9.81, pfa_range, slf_accel_norm) # The floor accelerations should be converted to g
        accel_floor_losses.append(accel_floor_loss)
    
    # Step 3: Get the total losses along the height of the building from acceleration-sensitive components
    accel_floor_losses = np.array(accel_floor_losses)
    accel_total_loss    = np.dot(pfa_weights, accel_floor_losses)

    # Step 4: Save results in dictionary
    results["drift_total_loss"].append(drift_total_loss)
    results["accel_total_loss"].append(accel_total_loss)
    results["total_loss"].append(drift_total_loss + accel_total_loss)
    results["IM"].append(ims['PGA'][i])

## Visualise the Total Losses of Drift- and Acceleration-Sensitive Nonstructural Components ## 

In [ ]:
# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # 1 row, 2 columns

# --- Drift-sensitive plot ---
axes[0].scatter(results["IM"], results["drift_total_loss"], color="tab:blue", alpha=0.7)
axes[0].set_xlabel("IM (PGA)")
axes[0].set_ylabel("Drift-Sensitive Total Loss")
axes[0].set_title("Drift-Sensitive Components")

# --- Acceleration-sensitive plot ---
axes[1].scatter(results["IM"], results["accel_total_loss"], color="tab:orange", alpha=0.7)
axes[1].set_xlabel("IM (PGA)")
axes[1].set_ylabel("Acceleration-Sensitive Total Loss")
axes[1].set_title("Acceleration-Sensitive Components")

# Improve layout
plt.tight_layout()
plt.show()

## Visualise the Combined Total Losses of Nonstructural Components and Fit Regression Model for Nonstructural Vulnerability ## 

In [ ]:
# Now, we fit a model to the total loss vs IM. We will explore various
# regression models suitable for values whose range vary between 0 and 1. 
# In this application, we refer to the lognormal CDF, generalised pareto
# distribution (GPD), the weibull function and the Papadopoulos [1] 
# nonlinear model. Eventually, we will select the best regression model 
# based on the goodness of fit parameter, SSE

# --- Load data ---
IM = np.array(results["IM"])
total_loss = np.array(results["total_loss"])

# Sort data by IM for fitting stability
sorted_idx = np.argsort(IM)
IM_sorted = IM[sorted_idx]
total_loss_sorted = total_loss[sorted_idx]

# Define regression models
models = {
    "Lognormal": {
        "func": lambda x, mean, sigma: lognorm.cdf(x, s=sigma, scale=np.exp(mean)),
        "p0": [0.0, 1.0]
    },
    "GPD": {
        "func": lambda x, c, loc, scale: genpareto.cdf(x, c, loc=loc, scale=scale),
        "p0": [0.1, 0.0, 1.0]
    },
    "Weibull": {
        "func": lambda x, a, b, c: a * (1 - np.exp(-((x / b) ** c))),
        "p0": [1.0, 1.0, 1.0]
    },
    "Papadopoulos": {
        "func": lambda x, a, b, c, d, e: e * (x ** a) / (b ** a + x ** a) + (1 - e) * (x ** c) / (d ** c + x ** c),
        "p0": [1.0, 1.0, 1.0, 1.0, 0.5]
    }
}

# Fit models 
sse_dict = {}
fitted_params = {}

for name, model in models.items():
    try:
        popt, _ = curve_fit(model["func"], IM_sorted, total_loss_sorted, p0=model["p0"], maxfev=10000)
        fitted_params[name] = popt
        fitted_curve = model["func"](IM_sorted, *popt)
        sse_dict[name] = np.sum((total_loss_sorted - fitted_curve) ** 2)
    except RuntimeError:
        print(f"Fit did not converge for {name}")
        fitted_params[name] = None
        sse_dict[name] = np.inf

# Rank models by SSE
ranked_models = sorted(sse_dict.items(), key=lambda x: x[1])
best_model = ranked_models[0][0]

# Create smooth IM grid for plotting
intensities = np.geomspace(0.05, 5, 50)

# Plot the scatter and the fitted regression models
plt.figure(figsize=(8,5))
plt.scatter(IM_sorted, total_loss_sorted, color='black', label='Data', alpha=0.7)

colors = {"Lognormal":"tab:blue", "GPD":"tab:green", "Weibull":"tab:orange", "Papadopoulos":"tab:red"}

for name, model in models.items():
    if fitted_params[name] is not None:
        curve = model["func"](IM_range, *fitted_params[name])
        label = f"{name} (best)" if name == best_model else name
        plt.plot(intensities, curve, color=colors[name], linewidth=2, label=label)

plt.xlabel("IM (PGA)")
plt.ylabel("Nonstructural Components Total Loss")
plt.title("Regression Fit of Models to IM vs Total Loss")
plt.legend()
plt.grid(True)
plt.xlim([0, 5])
plt.ylim([0, 1])
plt.show()

# Print table with short summary of goodness-of-fit 
table = pd.DataFrame(ranked_models, columns=["Model", "SSE"])
table["Rank"] = np.arange(1, len(table)+1)
print(table)

# NOTE: The nonstructural vulnerability derived above represents the total loss ratio 
# associated with nonstructural components conditioned on non-collapse of the structure.
# However, in vulnerability applications, it is necessary to condition the non-collapse 
# loss on the collapse fragility of the structure (i.e., due consideration of the probability 
# of collapse of the structure)

# Extract the non-conditioned losses associated with the best-fit model
best_curve_func = models[best_model]["func"]
best_curve_params = fitted_params[best_model]

nonconditioned_loss = best_curve_func(IM_range, *best_curve_params)

# Now `nonconditioned_loss` contains the fitted losses on the smooth IM_range
print("Best-fitting model:", best_model)
print("Nonconditioned losses (sampled on IM_range):")
print(nonconditioned_loss)

In [ ]:
# Let us consider that this case study structure has a median seismic intensity and 
# dispersion associated with complete damage of 2.0g of PGA and 0.3, respectively 

median_collapse = 2.0
dispersion_collapse = 0.3

# Let us now create the "collapse" fragility using OQ-VMTK's postprocessor module
collapse_fragility = pp.calculate_lognormal_fragility(median_collapse,
                                                      dispersion_collapse,
                                                      intensities=intensities)

# Plot the collapse fragility
plt.figure(figsize=(8,5))
plt.plot(intensities, collapse_fragility, color='black', alpha=0.7)
plt.xlabel('IM (PGA)')
plt.ylabel('Collapse Fragility')
plt.xlim([0,5])
plt.ylim([0,1])

In [ ]:
# Now, we condition the non-collapse (NC) nonstructural losses on collapse (C) via the following equation
# Loss = E[L,NC|IM]*(1-P(C|IM)) + P(C|IM)*1.00 where 1.0 corresponds to attaining a loss ratio of 1 (total replacement)

conditioned_loss = nonconditioned_loss*(1-collapse_fragility) + collapse_fragility*1.00

In [ ]:
# Visualise the non-collapse and conditioned loss model
plt.figure(figsize=(8,5))
plt.plot(intensities, nonconditioned_loss, color='blue', label = 'Non-Conditioned Loss')
plt.plot(intensities, conditioned_loss,    color='red',  label = 'Conditioned Loss')
plt.xlabel('IM (PGA)')
plt.ylabel('Nonstructural Components Total Loss')
plt.legend()
plt.grid('on', which = 'both')
plt.xlim([0,5])
plt.ylim([0,1])